## Using PSOpenAI to connect to OpenAI

In [ ]:
Request-ChatCompletion -Message "What is a splat?" | Select-Object -ExpandProperty Answer

In [ ]:
Request-ChatCompletion -Message "What is a splat?"

## PowerShell behind the scenes

In [ ]:
# Set the headers
$headers = @{
    "Content-Type"  = "application/json"
    "Authorization" = "Bearer $env:OPENAI_API_KEY"
}

# Prepare the messages array
$msgs = @(
    @{
        role    = "user"
        content = "What is a splat?"
    }
)

# Create the splat for Invoke-RestMethod
$splat = @{
    Uri     = "https://api.openai.com/v1/chat/completions"
    Method  = "Post"
    Headers = $headers
    Body    = @{
        model    = "gpt-4o"
        messages = $msgs
    } | ConvertTo-Json
}

$response = Invoke-RestMethod @splat

# Output the assistant's reply
$response.choices[0].message.content

In [ ]:
$response

### Narrow its context

In [32]:
# Set default values
$PSDefaultParameterValues["Invoke-RestMethod:Headers"] = $headers
$PSDefaultParameterValues["Invoke-RestMethod:Method"] = "POST"
$PSDefaultParameterValues["Invoke-RestMethod:Uri"] = "https://api.openai.com/v1/chat/completions"
$PSDefaultParameterValues["*:OutVariable"] = "response"

# Prepare the messages array
$msgs = @(
    @{
        role    = "system"
        content = "You are a PowerShell Expert."
    }
    @{
        role    = "user"
        content = "What is a splat?"
    }
)

# Prepare the body
$body = @{
    model    = "gpt-4o"
    messages = $msgs
} | ConvertTo-Json

$response = Invoke-RestMethod -Body $body

# Output the assistant's reply
$response.choices[0].message.content

In PowerShell, "splatting" is a technique used to pass a collection of parameter values to a command. This is particularly useful when a command has a large number of parameters or when you want to script in a more readable and maintainable way. Splatting uses a hash table or an array to bundle parameters together and then pass them to the command.

There are two main types of splatting in PowerShell:

1. **Hash Table Splatting:** This is used when you need to pass named parameters.

   ```powershell
   $params = @{
       Path = "C:\MyFolder"
       Filter = "*.txt"
       Recurse = $true
   }

   Get-ChildItem @params
   ```

   In this example, `@params` is a hash table that contains the parameters for `Get-ChildItem` cmdlet. The `@` character is used to splat the hash table.

2. **Array Splatting:** This is used when you need to pass positional parameters.

   ```powershell
   $params = "C:\MyFolder", "*.txt", $true

   Get-ChildItem @params
   ```

   In this example, `@params` is

## How do I know which models I can use?

In [29]:
(Invoke-RestMethod -Method Get -Uri https://api.openai.com/v1/models).data.id

gpt-4o-2024-08-06
gpt-4-1106-preview
chatgpt-4o-latest
tts-1-hd-1106
tts-1-hd
dall-e-2
gpt-4o-mini
text-embedding-3-large
gpt-4o-mini-2024-07-18
tts-1
tts-1-1106
gpt-4-0125-preview
gpt-3.5-turbo-0125
gpt-4-turbo-preview
gpt-3.5-turbo
whisper-1
gpt-4-turbo
gpt-4-turbo-2024-04-09
gpt-3.5-turbo-16k
text-embedding-3-small
gpt-3.5-turbo-1106
gpt-3.5-turbo-instruct-0914
gpt-4-0613
gpt-4
gpt-3.5-turbo-instruct
gpt-4o
babbage-002
davinci-002
dall-e-3
gpt-4o-2024-05-13
text-embedding-ada-002
ft:gpt-3.5-turbo-0613:rcr::9fXKjujH:ckpt-step-76
ft:gpt-3.5-turbo-0613:rcr::9fXKjY0j


## The magic 

In [30]:
Import-Module ./dbatools.ai.psd1

In [40]:
Import-DbaiFile -RequiredText "2019" -OutVariable response
$response
<#
$splat = @{
    Path            = './lib/immunization.pdf'
    JsonSchemaPath  = './lib/immunization.json'
    SqlInstance     = 'localhost'
    Database        = 'tempdb'
    Schema          = 'dbo'
    SystemMessage   = 'Convert text to structured data.'
}

Import-DbaiFile @splat
#>


ProcessedFile    : immunization.pdf
Markdown         : Below is the extracted text from the PDF document converted to Markdown format:
                   
                   ```
                   IMMUNIZATION RECORD FORM
                   
                   Name of Patient: _________________________________________________
                   Date of Birth: ________________
                   Patient ID or MRN #: _____________________________________________
                   Phone Number: ____________________________
                   
                   Vaccination Details:
                   
                   | Vaccine Name    | Date Given | Lot #       | Manufacturer | Administered By |
                   |-----------------|------------|-------------|--------------|-----------------|
                   | Hepatitis B     |            |             |              |                 |
                   | DTaP            |            |             |              |               

Error: Command failed: SubmitCode: Import-DbaiFile -RequiredText "2019" -OutVariable  ...

In [38]:
$response.SqlResults